# 16S rRNA gene amplicon survey - NASA

March 8, 2023

This notebook we will prepare and import Raymond Otoo's sequence data into QIIME 2, remove the primer sequences (if there are any), then denoise.

This data set was generated by [RTL Genomics](https://rtlgenomics.com/), with the [V3V4 primers (357wF assay)](https://static1.squarespace.com/static/5807c0ce579fb39e1dd6addd/t/63dd67d948c93362eea51cf7/1675454426339/Amplicon_Diversity_Assay_List_2019_V2.pdf).

Run this notebook using the `qiime2-amplicon-2023.9` or `qiime2-amplicon-2024.2` conda environment.

In [1]:
# Import common python functions
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [2]:
# I made the directories in the terminal before running this notebook using
# e.g. 'mkdir Processed'
mwd = '/home/mrobeson/projects/Antino_Allen'
wd = mwd + '/Processed/NASA'
rd = mwd + '/Raw.Data'
nd = mwd + '/Scripts.Notes'

metadata = wd + '/NASA-Metadata.tsv'

In [3]:
chdir(wd)
getcwd()

'/home/mrobeson/projects/Antino_Allen/Processed/NASA'

## Make manifest file and import

See:
https://www.linuxjournal.com/content/pattern-matching-bash

In [13]:
! unzip $rd/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024.zip -d $rd/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024

Archive:  /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024.zip
  inflating: /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-49-357wF-806R_R1.fastq  
  inflating: /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-49-357wF-806R_R2.fastq  
  inflating: /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-50-357wF-806R_R1.fastq  
  inflating: /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-50-357wF-806R_R2.fastq  
  inflating: /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-51-357wF-806R_R1.fastq  
  inflating: /home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-51-357wF-806R_R2.fastq  
  inflating: /home/mrobeson/project

In [22]:
%%bash

rd='/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024'
# F == full R1 file path
# DP == Directory path of F
# FFN == file name of F
# PRESAMPLENAME == prep for SAMPLE
# SAMPLE == Base sample file name
# R == full R2 file path

# remove manifest file if present
rm manifest.tsv

# make file and add header
touch manifest.tsv
echo -e "sample-id\tforward-absolute-filepath\treverse-absolute-filepath" >> manifest.tsv

# loop through and create file paths, append to manifest.txt file
for F in $(find $rd/ -name "*_R1.fastq"); do
        DP=${F%/*}
        FFN=${F##*/}
        #PRESAMPLENAME=${FFN%-357wF-806R*}
        #SAMPLE=${PRESAMPLENAME##*7148-}
        SAMPLE=${FFN%-357wF-806R*}
        R=$DP/${FFN%R1*}R2.fastq
        echo -e "$SAMPLE\t$F\t$R" >> manifest.tsv; 
done;


In [26]:
! cat manifest.tsv

sample-id	forward-absolute-filepath	reverse-absolute-filepath
7148-86	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-86-357wF-806R_R1.fastq	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-86-357wF-806R_R2.fastq
7148-76	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-76-357wF-806R_R1.fastq	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-76-357wF-806R_R2.fastq
7148-84	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-84-357wF-806R_R1.fastq	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-84-357wF-806R_R2.fastq
7148-70	/home/mrobeson/projects/Antino_Allen/Raw.Data/RTL-Data-03-04-2024/RTL-7148/Robeson_7148Raw02272024/7148-70-357wF-806R_R1.fastq	/home/mrobeson/pr

## import

In [24]:
! qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-format PairedEndFastqManifestPhred33V2 \
    --input-path manifest.tsv \
    --output-path paired-end-demux.qza

Imported manifest.tsv as PairedEndFastqManifestPhred33V2 to paired-end-demux.qza


In [25]:
! qiime demux summarize \
    --i-data paired-end-demux.qza \
    --o-visualization paired-end-demux.qzv

Saved Visualization to: paired-end-demux.qzv


## run cutadapt

We will use the following updated [V3V4 primer sequences](https://static1.squarespace.com/static/5807c0ce579fb39e1dd6addd/t/63dd67d948c93362eea51cf7/1675454426339/Amplicon_Diversity_Assay_List_2019_V2.pdf) from RTL Genomics.

Assay: 357wF 
- Fw (357wF) 5' - CCTACGGGNGGCWGCAG - 3'
- Rev (806R) 5' - GGACTACHVGGGTWTCTAAT - 3'

These primers typically amplify fragments of ~443 bases in length.

In [28]:
! qiime cutadapt trim-paired \
    --i-demultiplexed-sequences paired-end-demux.qza \
    --p-cores 8 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GGACTACHVGGGTWTCTAAT \
    --p-match-read-wildcards \
    --p-match-adapter-wildcards \
    --p-discard-untrimmed \
    --o-trimmed-sequences paired-end-demux-ca.qza \
    --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 8 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 1 -q 0,0 --quality-base 33 -o /home/mrobeson/tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-x0u65lwd/7148-49_20_L001_R1_001.fastq.gz -p /home/mrobeson/tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-x0u65lwd/7148-49_55_L001_R2_001.fastq.gz --front CCTACGGGNGGCWGCAG -G GGACTACHVGGGTWTCTAAT --match-read-wildcards --discard-untrimmed /home/mrobeson/tmp/qiime2/mrobeson/data/f8de8426-79c2-44d2-a509-8e6c680a85e3/data/7148-49_20_L001_R1_001.fastq.gz /home/mrobeson/tmp/qiime2/mrobeson/data/f8de8426-79c2-44d2-a509-8e6c680a85e3/data/7148-49_55_L001_R2_001.fastq.gz

This is cutadapt 4.5 with Python 3.8.15
Command line parameters: --cores 8 --error-rate 0.1 --times 1 --overlap 3 --minimum-le

In [29]:
! qiime demux summarize \
    --i-data paired-end-demux-ca.qza \
    --o-visualization paired-end-demux-ca.qzv

Saved Visualization to: paired-end-demux-ca.qzv


## Denoising

We'll try DADA2 and deblur.

### Try DADA2

[This thread](https://github.com/benjjneb/dada2/issues/1042#issuecomment-648400339) is quite informative on some DADA2 settings. Also see this [preprint](https://www.biorxiv.org/content/10.1101/081257v1.full). For more details see the [usearch uchime_denovo](https://www.drive5.com/usearch/manual/cmd_uchime3_denovo.html), [abundance skew](https://drive5.com/usearch/manual/abundance_skew.html), and the [chimeras](https://drive5.com/usearch/manual/chimeras.html) documentation. I should probably should set `min-fold-parent-over-abundance` to `8` or `16`, rather than `2`? In the thread, Callahan seems to suggest `8` might be a good balance.

Try a few different trunc-f and trunc-r settings, with the `min-fold-parent-over-abundance` setting:

- 266, 175 w/ min-fold-parent-over-abundance 8
- xxx, yyy w/ min-fold-parent-over-abundance zzz


In [ ]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs paired-end-demux-ca.qza \
    --p-trunc-len-f 266 \
    --p-trunc-len-r 175 \
    --p-trim-left-f 0 \
    --p-trim-left-r 0 \
    --p-pooling-method 'pseudo' \
    --p-chimera-method 'pooled' \
    --p-min-fold-parent-over-abundance 8 \
    --p-n-threads 8 \
    --o-table  dada2-pe-table.qza \
    --o-representative-sequences dada2-pe-repseqs.qza \
    --o-denoising-stats dada2-pe-stats.qza \
    --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /home/mrobeson/tmp/tmp3xeu59we/forward --input_directory_reverse /home/mrobeson/tmp/tmp3xeu59we/reverse --output_path /home/mrobeson/tmp/tmp3xeu59we/output.tsv.biom --output_track /home/mrobeson/tmp/tmp3xeu59we/track.tsv --filtered_directory /home/mrobeson/tmp/tmp3xeu59we/filt_f --filtered_directory_reverse /home/mrobeson/tmp/tmp3xeu59we/filt_r --truncation_length 266 --truncation_length_reverse 175 --trim_left 0 --trim_left_reverse 0 --max_expected_errors 2.0 --max_expected_errors_reverse 2.0 --truncation_quality_score 2 --min_overlap 12 --pooling_method pseudo --chimera_method pooled --min_parental_fold 8 --allow_one_off False --num_threads 8 --learn_min_reads 1000000

Warning message:
package ‘optparse’ was built un

In [ ]:
# summarize denoising stats
! qiime metadata tabulate \
    --m-input-file dada2-pe-stats.qza  \
    --o-visualization dada2-pe-stats.qzv

In [ ]:
# summarize ESV table
! qiime feature-table summarize \
    --i-table dada2-pe-table.qza \
    --o-visualization dada2-pe-table.qzv \
    --m-sample-metadata-file $metadata

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data dada2-pe-repseqs.qza \
    --o-visualization dada2-pe-repseqs.qzv

### Try deblur

Generally following [this approach](https://docs.qiime2.org/2024.2/tutorials/read-joining/).

In [ ]:
! qiime vsearch merge-pairs \
    --i-demultiplexed-seqs paired-end-demux-ca.qza \
    --p-threads 8 \
    --o-merged-sequences paired-end-demux-ca-merged.qza \
    --verbose

In [ ]:
! qiime demux summarize \
    --i-data paired-end-demux-ca-merged.qza \
    --p-n 20000 \
    --o-visualization paired-end-demux-ca-merged.qzv

In [ ]:
! qiime quality-filter q-score \
    --i-demux paired-end-demux-ca-merged.qza \
    --o-filtered-sequences paired-end-demux-ca-merged-filt.qza \
    --o-filter-stats paired-end-demux-ca-merged-stats.qza

In [ ]:
! qiime demux summarize \
    --i-data paired-end-demux-ca-merged-filt.qza \
    --p-n 20000 \
    --o-visualization paired-end-demux-ca-merged-filt.qzv

In [ ]:
! qiime metadata tabulate \
    --m-input-file paired-end-demux-ca-merged-stats.qza  \
    --o-visualization paired-end-demux-ca-merged-stats.qzv

In [ ]:
# Use SILVA as ref
#    `qiime deblur denoise-other`
#    silva files are located here: https://docs.qiime2.org/2023.9/data-resources/
#    I used my own...
! qiime deblur denoise-other \
    --i-demultiplexed-seqs paired-end-demux-ca-merged-filt.qza  \
    --i-reference-seqs /home/mrobeson/dbs/qiime.taxonomy/SILVA/q2-2023.9/without-species/silva-138-1-ssu-nr99-seqs-dr-uniq-cln-filt.qza \
    --p-trim-length 400 \
    --p-jobs-to-start 8 \
    --o-table deblur-pe-table.qza \
    --o-representative-sequences deblur-pe-repseqs.qza \
    --o-stats deblur-pe-stats.qza \
    --verbose

In [ ]:
! qiime deblur visualize-stats \
    --i-deblur-stats deblur-pe-stats.qza \
    --o-visualization deblur-pe-stats.qzv

In [ ]:
! qiime feature-table summarize \
    --i-table deblur-pe-table.qza \
    --o-visualization deblur-pe-table.qzv \
    --m-sample-metadata-file $metadata

! qiime feature-table tabulate-seqs \
    --i-data deblur-pe-repseqs.qza \
    --o-visualization deblur-pe-repseqs.qzv

In [ ]:
! qiime vsearch uchime-denovo \
    --i-sequences deblur-pe-repseqs.qza \
    --i-table deblur-pe-table.qza  \
    --o-chimeras deblur-pe-repseqs-chimeras.qza \
    --o-nonchimeras deblur-pe-repseqs-nochimeras.qza \
    --o-stats deblur-pe-repseqs-uchime-denovo-stats.qza\
    --verbose

In [ ]:
! qiime deblur visualize-stats \
    --i-deblur-stats deblur-pe-repseqs-uchime-denovo-stats.qza \
    --o-visualization deblur-pe-repseqs-uchime-denovo-stats.qzv \

In [ ]:
! qiime feature-table filter-features \
    --i-table deblur-pe-table.qza \
    --m-metadata-file deblur-pe-repseqs-nochimeras.qza \
    --o-filtered-table deblur-pe-table-nochimeras.qza

In [ ]:
! qiime feature-table summarize \
    --i-table deblur-pe-table-nochimeras.qza \
    --o-visualization deblur-pe-table-nochimeras.qzv \
    --m-sample-metadata-file $metadata

! qiime feature-table tabulate-seqs \
    --i-data deblur-pe-repseqs-nochimeras.qza \
    --o-visualization deblur-pe-repseqs-nochimeras.qzv